In [1]:
import os
import shutil
import pandas as pd

In [6]:
excel_path = "blast/MasterlistAug30-2017.xlsx"
img_root   = "blast/HMC/Images"
dest_root  = "blast/HMC" 

In [7]:
df = pd.read_excel(excel_path)

In [8]:
df

,Unnamed: 0,Unnamed: 1,File Name,GRADE,Expansion,ICM,TE,Outcome,Acquisition Year
0,NaN,NaN,NaN,According to the Gardner's Method,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Blast_PCRM_R12-0137,4AA,4.0,A,A,1.0,2012
2,NaN,NaN,Blast_PCRM_R12-0160,4AA,4.0,A,A,1.0,2012
3,NaN,NaN,Blast_PCRM_R12-0173a,3AB,3.0,A,B,0.0,2012
4,NaN,NaN,Blast_PCRM_R12-0173b,2AA,2.0,A,A,0.0,2012
...,...,...,...,...,...,...,...,...,...
245,NaN,NaN,Blast_PCRM_d5.17,2BC,2.0,B,C,2.0,unknown
246,NaN,NaN,Blast_PCRM_d5.22,2AB,2.0,A,B,2.0,unknown
247,NaN,NaN,Blast_PCRM_d5.24,4AA,4.0,A,A,2.0,unknown
248,NaN,NaN,Blast_PCRM_d5.26,4BA,4.0,B,A,2.0,unknown


In [10]:
for _, row in df.iterrows():
    fname   = str(row["File Name"]).strip()   # 예: Blast_PCRM_R12-0137
    outcome = row["Outcome"]                  # 0, 1, 2(unknown)

    # Outcome 0/1만 처리
    if outcome not in (0, 1):
        continue

    # 원본 · 대상 경로 구성
    src  = os.path.join(img_root, f"{fname}.BMP")
    dest_dir = os.path.join(dest_root,
                            "success" if outcome == 1 else "failure")
    os.makedirs(dest_dir, exist_ok=True)
    dst = os.path.join(dest_dir, f"{fname}.BMP")

    # 파일 이동
    try:
        shutil.move(src, dst)
    except FileNotFoundError:
        print(f"[WARN] 원본 없음: {src}")

print("완료!")

완료!


In [15]:
from pathlib import Path
root_dir = Path("blast/HMC/")
success_dir = root_dir / "success"
failure_dir = root_dir / "failure"

def count_files(directory: Path) -> int:
    return sum(1 for p in directory.iterdir() if p.is_file())

n_success = count_files(success_dir)
n_failure = count_files(failure_dir)

print(f"success 폴더 파일 수  : {n_success}")
print(f"failure 폴더 파일 수  : {n_failure}")

success 폴더 파일 수  : 96
failure 폴더 파일 수  : 75


In [16]:
src_root  = Path("blast/HMC/")
dst_root  = Path("blast/split")
train_ratio, val_ratio = 0.7, 0.2
seed = 42

In [18]:
import random
random.seed(seed)

def split_and_copy(class_name: str):
    src_dir = src_root / class_name          # HMC/success 또는 HMC/failure
    files   = [p for p in src_dir.iterdir() if p.is_file()]

    # 1) 셔플
    random.shuffle(files)

    # 2) 개수 계산
    n_total = len(files)
    n_train = round(n_total * train_ratio)
    n_val   = round(n_total * val_ratio)
    n_test  = n_total - n_train - n_val      # 잔여분은 test로

    splits = {
        "train": files[:n_train],
        "val":   files[n_train:n_train+n_val],
        "test":  files[n_train+n_val:],
    }

    # 3) 복사
    for split_name, split_files in splits.items():
        dst_dir = dst_root / class_name / split_name
        dst_dir.mkdir(parents=True, exist_ok=True)

        for f in split_files:
            shutil.copy2(f, dst_dir / f.name)

        print(f"[{class_name}/{split_name}] {len(split_files)}개 복사 완료")

# 실행
for cls in ("success", "failure"):
    split_and_copy(cls)

print("✅ 전체 복사 완료!")

[success/train] 67개 복사 완료
[success/val] 19개 복사 완료
[success/test] 10개 복사 완료
[failure/train] 52개 복사 완료
[failure/val] 15개 복사 완료
[failure/test] 8개 복사 완료
✅ 전체 복사 완료!
